In [1]:
import tensorflow as tf 
import numpy as np
import os 

2023-04-01 11:52:53.883498: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-01 11:52:53.930905: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-01 11:52:53.931656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 11:52:54.739943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
text = ""
with open("../Data/Tweets.txt", "r") as f:
    text = f.read()

text = text.replace('\n', ' ')

In [3]:
vocabfile = text
# Create a vocabulary of characters
vocab = sorted(set(vocabfile.split(' ')))

print(vocab)

['', '"a', '"akala', '"alam', '"all', '"ang', '"anger"', '"ano?', '"aquino', '"bakit', '"be', '"confident', '"crossini', '"don\'t', '"flexible', '"get', '"happy', '"i', '"i\'m', '"ikaw', '"kulang', '"liwanag"', '"lover"', '"nasasakal', '"nothing"', '"para', '"picturan', '"piliin', '"please', '"pm', '"sana', '"seriously,', '"shithole"', '"special"', '"tangina', '"the', '"things"', '"think', '"tulungan', '"wala', '"what', '"what\'s', '"why?"', '"wouldn\'t', '#bdonomura', '#buhatbayag', '#chu2koi', '#nowplaying', '#oregairu', '#spotifywrapped', '#sub2pewds', '#subscribetopewdiepie', '#俺ガイル', '#祝俺ガイル第3期', '%', '&amp;', '&lt;3', "'ako'", "'autshatfisisciarnme", "'bout", "'could", "'di", "'em", "'healthy", '\'ko."', '\'tayo\'?"', "'to", "'to.", '\'tomozaki-kun\'"', "'tong", "'wmpe/adil]3willsm🌎hurjyb.", '(yes)', '*bought', '*have', '*mind', '*still', ',', '-', '-alanis,', '-zero-', '.', '...', '._.', '0.2332', '0.2834', '0.5', '1', '1.', '10', '12', '140', '167', '2', '2.', '2.7875', '2019',

In [4]:

# Convert to index 
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [5]:
len(vocab)

text_to_int = np.array([char2idx[x] for x in text.split(' ')])

# Parse to train per sequence
sequence_length = 20
examples_per_epoch = len(text.split(' ')) //sequence_length

# Slice dataset as a tensor
char_dataset = tf.data.Dataset.from_tensor_slices(text_to_int)

# Cut for desirable length 
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(' '.join(idx2char[item.numpy()])))

# Function to split dataset {FOR TRAINING}: input and next target character
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Example Training
for iExample, tExample in dataset.take(1):
    print('Input Data: ', repr(' '.join(idx2char[iExample.numpy()])))
    print('Target Data: ', repr(' '.join(idx2char[tExample.numpy()])))
    for i, (input_idx, target_idx) in enumerate(zip(iExample[:4], tExample[:4])):
        print("Step {:4d}".format(i))
        print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
        print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


'will create llm for my tweets  salamat sa disappointment @greyunitato  salamat sa laging pag disappoint  2 am still'
'no sign  the usual. nakalimutan na may jowa. thank you. will do the same to you.  have a social'
'life just to spite  we angry, not sad  pretend that errthings okay  pretend like we okay  damn.'
'i’m getting tired of solitude but whatever  let’s bottle shit up and see how long we can stay sane! :)'
' goddamnit, i need a safe space  note to self: inarte = saloobin.  let me just mark this day'
Input Data:  'will create llm for my tweets  salamat sa disappointment @greyunitato  salamat sa laging pag disappoint  2 am'
Target Data:  'create llm for my tweets  salamat sa disappointment @greyunitato  salamat sa laging pag disappoint  2 am still'
Step    0
  input: 2425 ('will')
  expected output: 501 ('create')
Step    1
  input: 501 ('create')
  expected output: 1293 ('llm')
Step    2
  input: 1293 ('llm')
  expected output: 758 ('for')
Step    3
  input: 758 ('for')
  expec

2023-04-01 11:52:56.858194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [7923]
	 [[{{node Placeholder/_0}}]]
2023-04-01 11:52:56.899700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [7923]
	 [[{{node Placeholder/_0}}]]


In [6]:
# Initialize size of batch for training: The input node layer
batch_size = 8
steps_per_epoch = examples_per_epoch//batch_size

# Size of buffer
buffer_size = 1000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

vocab_size = len(vocab)
# 1st Layer of hidden Node
embedding_dim = 256 
# Recursive Layer 
rnn_units = 1024

# Initialize RNN Layer if for GPU or CPU
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

# Build model for initializing
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                  batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
           return_sequences=True,
           recurrent_initializer='glorot_uniform',
           stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

# Building Model
model = build_model(vocab_size=len(vocab),
                   embedding_dim=embedding_dim,
                   rnn_units=rnn_units,
                   batch_size=batch_size)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-04-01 11:52:57.284231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-01 11:52:57.286116: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-01 11:52:57.287356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [7]:
# Model summary
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
model.summary()

# Samples 
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

(8, 20, 2528) # (batch_size, sequence_length, vocab_size)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (8, None, 256)            647168    
                                                                 
 gru (GRU)                   (8, None, 1024)           3938304   
                                                                 
 dense (Dense)               (8, None, 2528)           2591200   
                                                                 
Total params: 7,176,672
Trainable params: 7,176,672
Non-trainable params: 0
_________________________________________________________________


2023-04-01 11:52:57.526226: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [7923]
	 [[{{node Placeholder/_0}}]]
2023-04-01 11:52:57.526610: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [7923]
	 [[{{node Placeholder/_0}}]]


In [8]:
# Calculates Loss
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction Shape: ", example_batch_predictions.shape,"# (batch_size, sequence_length, vocab_size)")
print("Scalar Loss: ", example_batch_loss.numpy().mean())

# Compile model 
model.compile(optimizer=tf.optimizers.Adam(), loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Model Training 
# Callbacks are used to refit shape of model
history = model.fit(dataset.repeat(), epochs=50, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Prediction Shape:  (8, 20, 2528) # (batch_size, sequence_length, vocab_size)
Scalar Loss:  7.835019
Epoch 1/50


2023-04-01 11:52:58.154678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [7923]
	 [[{{node Placeholder/_0}}]]
2023-04-01 11:52:58.155075: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [7923]
	 [[{{node Placeholder/_0}}]]
2023-04-01 11:52:58.334443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' wi

49/49 [==============================] - 17s 327ms/step - loss: 7.4674
Epoch 2/50
49/49 [==============================] - 16s 330ms/step - loss: 6.4544
Epoch 3/50
49/49 [==============================] - 16s 325ms/step - loss: 5.9704
Epoch 4/50
49/49 [==============================] - 16s 327ms/step - loss: 5.4784
Epoch 5/50
49/49 [==============================] - 16s 332ms/step - loss: 4.9378
Epoch 6/50
49/49 [==============================] - 17s 340ms/step - loss: 4.4296
Epoch 7/50
49/49 [==============================] - 16s 331ms/step - loss: 3.7844
Epoch 8/50
49/49 [==============================] - 16s 324ms/step - loss: 3.0680
Epoch 9/50
49/49 [==============================] - 16s 325ms/step - loss: 2.3626
Epoch 10/50
49/49 [==============================] - 16s 327ms/step - loss: 1.7096
Epoch 11/50
49/49 [==============================] - 16s 324ms/step - loss: 1.2090
Epoch 12/50
49/49 [==============================] - 16s 332ms/step - loss: 0.8674
Epoch 13/50
49/49 [=====

In [16]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

2023-04-01 12:13:26.625254: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-01 12:13:26.627072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-01 12:13:26.628338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [17]:
model.save('../Model/twitter.word.keras')

In [18]:
def generate_text(model, start_string):
  
  num_generate = 50
  
  input_eval = [char2idx[s] for s in start_string.split(' ')]
  input_eval = tf.expand_dims(input_eval, 0)
  
  print(input_eval)
  
  text_generated = []
  
  temperature = 1.0
  
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    
    predictions = tf.squeeze(predictions, 0)
    
    predictions = predictions/ temperature
    
    predicted_id = tf.compat.v1.multinomial(predictions, num_samples=1)[-1,0].numpy()
    
    input_eval = tf.expand_dims([predicted_id],0)
    
    text_generated.append(idx2char[predicted_id])
    
  return (start_string + ' '.join(text_generated))

In [31]:
print(generate_text(model, start_string=u""))

tf.Tensor([[0]], shape=(1, 1), dtype=int32)
roses are red; violets are blue;  on september 27;  i'm going to you.  "ang childish mo" oh my fuckin god  i knew getting tired of solitude but whatever  let’s bottle shit up and see how long we can stay sane! :)  tangina ang sakit


In [24]:
loaded_model = tf.keras.saving.load_model("../Model/twitter.keras")

2023-04-01 12:14:17.997959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-01 12:14:17.999733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-01 12:14:18.000739: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [26]:
print(generate_text(loaded_model, start_string=u"re"))

KeyError: 're'